In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt

n = 3
N = 2**n
solution = '101'
def grover_oracle(n, solution):
    qc = QuantumCircuit(n)
    for qubit, bit in enumerate(reversed(solution)):
        if bit == '0':
            qc.x(qubit)
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1)  # Multi-controlled X (Toffoli)
    qc.h(n - 1)
    for qubit, bit in enumerate(reversed(solution)):
        if bit == '0':
            qc.x(qubit)
    oracle_gate = qc.to_gate()
    oracle_gate.name = "Oracle"
    return oracle_gate
def grover_diffuser(n):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1) 
    qc.h(n - 1)
    qc.x(range(n))
    qc.h(range(n))
    diffuser_gate = qc.to_gate()
    diffuser_gate.name = "Diffuser"
    return diffuser_gate


# Number of iterations
iterations = int(np.floor((np.pi / 4) * np.sqrt(N)))

# Prepare simulator with statevector saving
backend = AerSimulator(method='statevector')

probs = []

for r in range(iterations + 1):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    for _ in range(r):
        qc.append(grover_oracle(n, solution), range(n))
        qc.append(grover_diffuser(n), range(n))
    
    # Save statevector after the circuit
    qc.save_statevector()
    
    transpiled = transpile(qc, backend)
    result = backend.run(transpiled).result()
    state = result.data(0)['statevector']

    # index of '101'
    idx = int(solution, 2)
    amp = state[idx]
    probs.append(abs(amp)**2)

qc.measure(range(n), range(n))

qc.draw('mpl')
backend = AerSimulator()
shots = 2048

transpiled_qc = transpile(qc, backend)
# plotting histogram 
result = backend.run(transpiled_qc, shots=shots).result()
counts = result.get_counts(transpiled_qc)
print(counts)
plot_histogram(counts)

# Plot probability vs iterations
plt.plot(list(range(iterations + 1)), probs, marker='o')
plt.xlabel("Grover iterations")
plt.ylabel(f"Probability of |{solution}⟩")
plt.title("Target-State Probability vs Grover Iterations")
plt.grid(True)
plt.show()

#bonus question 1:
# for this circuit to work for any other 3 qubit state just replace the solution to the wanted state
#bonus question 2:
# for this to work for any 4 qubit system change n to be 4


In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
n = 3  
N = 2 ** n

def grover_diffuser(n):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n - 1)
    qc.mcx(list(range(n - 1)), n - 1) 
    qc.h(n - 1)
    qc.x(range(n))
    qc.h(range(n))
    diffuser_gate = qc.to_gate()
    diffuser_gate.name = "Diffuser"
    return diffuser_gate
iterations = int(np.floor((np.pi*1.0) / 4 * np.sqrt(N))) #assumed M to be 1 as number of possible cases here at only 2^3=8
qc = QuantumCircuit(n, n)

qc.h(range(n))

from qiskit.circuit.library import PhaseOracle
oracle = PhaseOracle('(x1 or not x2 or x3) and (not x1 or x2 or x3)') #the inbuilt phase oracle function basically marks the "wanted" solution by flipping its phase through a command given in terms of a boolean clause
diffuser = grover_diffuser(n)

for _ in range(iterations):
    qc.append(oracle, range(n))
    qc.append(diffuser, range(n))

qc.measure(range(n), range(n))
bitstr=max(counts, key=counts.get)
temp=[]
temp = [bit == '1' for bit in bitstr]  # will directly interpret bit 1 as true and bit 0 as false of all the bits in the bitstring in the list
x1 = temp[0]
x2 = temp[1]
x3 = temp[2]
if ((x1 or not x2 or x3) and (not x1 or x2 or x3)){
    print("the measured outcome does satisfy the initial SAT condition")
}
else{
    print("the measured outcome does NOT satisfy the initial SAT condition")
}
qc.draw('mpl')
from qiskit import transpile

backend = AerSimulator()
shots = 2048

transpiled_qc = transpile(qc, backend)

result = backend.run(transpiled_qc, shots=shots).result()
counts = result.get_counts(transpiled_qc)
print(counts)
plot_histogram(counts)

# bonus question 1:
# if i want to extend this SAT problem for more variables i will accordingly alter the value of n(number of bits) and the boolean expression as well
# bonus question 2:
# I'm a bit confused as to how can i implement and analyze M in the program will get back to you for the same